In [3]:
import os
import json
import time
import datetime
import requests

import schedule
from web3 import Web3

from apiclient import discovery
from google.oauth2 import service_account
from pycoingecko import CoinGeckoAPI


In [6]:
headers = {
    'accept': '*/*',
}

total_tvl = {}
for dex in ['uniswap', 'sushiswap', 'balancer', 'curve', 'trader-joe', 'spookyswap', 'solidly']:
    response = requests.get(f'https://api.llama.fi/protocol/{dex}', headers=headers)
    r = response.json()
    dex_tvl = {}
    for chain in r["chainTvls"]:
        tvl = {}
        if chain in ['Ethereum', 'Avalanche', 'Arbitrum', 'Polygon', 'Fantom']:
            for info in r["chainTvls"][chain]['tvl']:
                day, usd = datetime.datetime.utcfromtimestamp(info['date']).strftime('%Y-%m-%d'), info['totalLiquidityUSD']
                if day in tvl:
                    tvl[day]=tvl[day] + usd
                else:
                    tvl[day]=usd
            dex_tvl[chain] = tvl
    total_tvl[dex] = dex_tvl

In [18]:
sheets_values = []
for dex_name, dex_info in total_tvl.items():
    for chain_name, chain_info in dex_info.items():
        for day, tvl in chain_info.items():
            sheets_values.append([dex_name, chain_name, day, tvl])

In [21]:
def check_refresh(spreadsheet_id, sheet, range_name):
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    return result.get('values',[])[0][0], result.get('values',[])[1][0]


scopes = ["https://www.googleapis.com/auth/spreadsheets"]
secret_file = os.path.join(os.getcwd(), 'client_secret.json')
spreadsheet_id = "1cTYr0Kimk2O29t136BaKaePqqMEsb4xTojdTsMBZf3A"
#os.environ['LIQUIDITY_SPREADSHEET_ID']

In [24]:
try:    
    credentials = service_account.Credentials.from_service_account_file(secret_file, scopes=scopes)
    service = discovery.build('sheets', 'v4', credentials=credentials)
    sheet = service.spreadsheets()

    data = {'values' : sheets_values}
    sheet.values().update(spreadsheetId=spreadsheet_id, body=data, range=f'TVL!B2:E{len(sheets_values)+1}', valueInputOption='USER_ENTERED').execute()

except OSError as e:
    print (e)

In [ ]:
def run_graph_query(graph, query):
    request = requests.post(graph, json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

lp_pairs_query = """
{
  lpPairs(first: 1000) {
    id
    name
  }
}
"""

endpoints = [
    'https://api.thegraph.com/subgraphs/name/drondin/olympus-liquidity-ethereum',
    'https://api.thegraph.com/subgraphs/name/drondin/olympus-liquidity-avalanche'
]

liq = {}

for endpoint in endpoints:
  lp_pairs = run_graph_query(endpoint, lp_pairs_query)

  for lp in lp_pairs["data"]["lpPairs"]:
    
    pair_volume_query = """
    {
      dailyVolumes(orderBy: timestamp, orderDirection:desc, first:30) {
        volume
        timestamp
        lp {
          name
          liquidity
        }
      }
    }
    """
    pair_volume_daily = run_graph_query(endpoint, pair_volume_query)

    lp_volume = 0
    lp_liq_sum = 0
    lp_liq_max = 0

    for day in pair_volume_daily["data"]["dailyVolumes"]:
      lp_volume += float(day["volume"])
      lp_liq_sum += float(day["lp"]["liquidity"])
      lp_liq_max = max(float(day["lp"]["liquidity"]), lp_liq_max)
      print(lp['name'], lp['id'], datetime.datetime.utcfromtimestamp(int(day['timestamp'])).strftime('%Y-%m-%d'), float(day["lp"]["liquidity"]))